# EfficientNet PEFT on Cifar-10 with Cerebros

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Colab\ Notebooks/cerebros-core-algorithm-alpha
import sys
sys.path.insert(0, '../..')
!pip install -r requirements.txt

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/cerebros-core-algorithm-alpha
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.9/92.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 490.0/490.0 kB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.datasets import cifar10, cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Resizing, Lambda, Flatten, Dense
import pandas as pd
import numpy as np
from cerebros.simplecerebrosrandomsearch.simple_cerebros_random_search\
    import SimpleCerebrosRandomSearch
import pendulum
from cerebros.units.units import DenseUnit
from cerebros.denseautomlstructuralcomponent.dense_automl_structural_component\
    import zero_7_exp_decay, zero_95_exp_decay, simple_sigmoid
from ast import literal_eval

## Fine-tuning some layers of EfficientNet from scratch

This is done for comparison only, and may be skipped. This section provides no information on Cerebros' efficiency

Download EfficientNet (v.2, small model) with Imagenet weights (1000 classes)

In [3]:
enet = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
    include_preprocessing=True
)

87549136/87549136 [==============================] - 1s 0us/step


In [4]:
enet.summary()

Model: "efficientnetv2-s"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 384, 384, 3)]        0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, 384, 384, 3)          0         ['input_1[0][0]']             
                                                                                                  
 stem_conv (Conv2D)          (None, 192, 192, 24)         648       ['rescaling[0][0]']           
                                                                                                  
 stem_bn (BatchNormalizatio  (None, 192, 192, 24)         96        ['stem_conv[0][0]']           
 n)                                                                                

Make all layers untrainable except for the very last convolutional layer

In [5]:
for layer in enet.layers:
    layer.trainable = False
enet.layers[-13].trainable = True
enet.layers[-12].trainable = True
enet.layers[-6].trainable  = True

Download and prepare Cifar-10 data

In [6]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 5s 0us/step


In [7]:
y_train_cat = to_categorical(y_train, 1000)
y_test_cat = to_categorical(y_test, 1000)

Resizing images to meet EfficientNet's input shape

In [8]:
def resize(x):
    return tf.image.resize(x,size=(384,384),method='bilinear')

Modify the model

In [9]:
input_shape = (32,32,3)

In [10]:
input_layer = Input(shape=input_shape)
prep = Lambda(resize)(input_layer)
out = enet(prep)
enet_mod = Model(inputs=input_layer, outputs=out)

In [11]:
enet_mod.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(),
                 metrics=[tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='top_1_categorical_accuracy')])

Try to fit it on Cifar-10 data and then evaluate (this will be efficient enough if trained on the complete dataset ...)

In [12]:
enet_mod.fit(X_train, y_train_cat)

1563/1563 [==============================] - 142s 76ms/step - loss: 0.4024 - top_1_categorical_accuracy: 0.8732


In [13]:
enet_mod.evaluate(X_test, y_test_cat)

313/313 [==============================] - 26s 73ms/step - loss: 0.2647 - top_1_categorical_accuracy: 0.9133


[0.26468804478645325, 0.9132999777793884]

## PEFT with Cerebros

Download Cifar-10 data

In [14]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [15]:
input_shape   = (32,32,3)
INPUT_SHAPES  = [input_shape]
OUTPUT_SHAPES = [10]

Subsampling a small balanced set of samples (with of without shuffling) from the train dataset and train labels

In [16]:
def subsample_train(X_train, y_train, num_samples):
    #
    X_sub = []
    y_sub = []
    #
    assert 1 <= num_samples <= 5000
    #
    for cat in range(10):
        #
        ind, _ = np.where(y_train==cat)
        #
        X_cat = X_train[ind]
        X_cat = X_cat[:num_samples]
        y_cat = y_train[ind]
        y_cat = y_cat[:num_samples]
        #
        X_sub += [X_cat]
        y_sub += [y_cat]
    #
    X_sub = np.vstack(X_sub)
    y_sub = np.vstack(y_sub)
    #
    assert X_sub.shape[0] == y_sub.shape[0]
    #
    ind = np.arange(X_sub.shape[0])
    np.random.shuffle(ind)
    #
    return X_sub[ind], y_sub[ind]

We take only 200 samples in each category (out of 5000)

In [17]:
num_samples = 200
X_sub, y_sub = subsample_train(X_train, y_train, num_samples)

Preparing tensors for the training set and labels

In [18]:
training_x   = [tf.constant(X_sub)]
y_train_cat  = to_categorical(y_sub, 10)
train_labels = [tf.constant(y_train_cat)]

Donwloading EfficientNet (v.2, small)

In [19]:
enet = tf.keras.applications.efficientnet_v2.EfficientNetV2S(
    include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation='softmax',
    include_preprocessing=True
)

Resizing images to meet EfficientNet's input shape

In [20]:
def resize(x):
    return tf.image.resize(x,size=(384,384),method='bilinear')

Make only the last convolutional layer trainable

In [21]:
for layer in enet.layers:
    layer.trainable = False
enet.layers[-13].trainable = True
enet.layers[-12].trainable = True
enet.layers[-6].trainable  = True

Preparing the base model for Cerebros search

In [22]:
enet_io = Model(inputs=enet.layers[0].input,
                outputs=enet.layers[-3].output)

In [23]:
input_layer = Input(shape=input_shape)
prep = Lambda(resize)(input_layer)
out = Flatten()(enet_io(prep))
base_mod = Model(inputs=input_layer, outputs=out)

In [24]:
activation = 'swish'
predecessor_level_connection_affinity_factor_first = 2.0
predecessor_level_connection_affinity_factor_main = 0.97
max_consecutive_lateral_connections = 5
p_lateral_connection = 0.97
num_lateral_connection_tries_per_unit = 2
learning_rate = 0.001
epochs = 5  # [1, 100]
batch_size = 20
maximum_levels = 6  # [3,7]
maximum_units_per_level = 5  # [2,10]
maximum_neurons_per_unit = 4  # [2,20]

In [25]:
# Final training task
TIME = pendulum.now(tz='America/New_York').__str__()[:16]\
    .replace('T', '_')\
    .replace(':', '_')\
    .replace('-', '_')
#
PROJECT_NAME = f'{TIME}_cerebros_auto_ml_test_cifar10_efficientnet'
#
meta_trial_number = 42
#
cerebros_automl = SimpleCerebrosRandomSearch(
    unit_type=DenseUnit,
    input_shapes=INPUT_SHAPES,
    output_shapes=OUTPUT_SHAPES,
    training_data=training_x,
    labels=train_labels,
    validation_split=0.2,
    direction='maximize',
    metric_to_rank_by="val_top_1_categorical_accuracy",
    minimum_levels=2,
    maximum_levels=maximum_levels,
    minimum_units_per_level=1,
    maximum_units_per_level=maximum_units_per_level,
    minimum_neurons_per_unit=1,
    maximum_neurons_per_unit=maximum_neurons_per_unit,
    activation=activation,
    final_activation='softmax',
    number_of_architecture_moities_to_try=3,
    number_of_tries_per_architecture_moity=2,
    minimum_skip_connection_depth=1,
    maximum_skip_connection_depth=7,
    predecessor_level_connection_affinity_factor_first=predecessor_level_connection_affinity_factor_first,
    predecessor_level_connection_affinity_factor_first_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_main=predecessor_level_connection_affinity_factor_main,
    predecessor_level_connection_affinity_factor_main_rounding_rule='ceil',
    predecessor_level_connection_affinity_factor_decay_main=zero_7_exp_decay,
    seed=8675309,
    max_consecutive_lateral_connections=max_consecutive_lateral_connections,
    gate_after_n_lateral_connections=3,
    gate_activation_function=simple_sigmoid,
    p_lateral_connection=p_lateral_connection,
    p_lateral_connection_decay=zero_95_exp_decay,
    num_lateral_connection_tries_per_unit=num_lateral_connection_tries_per_unit,
    learning_rate=learning_rate,
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[tf.keras.metrics.TopKCategoricalAccuracy(
                k=1, name='top_1_categorical_accuracy')
             ],
    epochs=epochs,
    project_name=f"{PROJECT_NAME}_meta_{meta_trial_number}",
    model_graphs='model_graphs',
    batch_size=batch_size,
    meta_trial_number=meta_trial_number,
    base_models=[base_mod])

In [26]:
%%time
result = cerebros_automl.run_random_search()

SimpleCerebrosRandomSearch.input_shapes: [(32, 32, 3)]
nan
>nnf>ceil
k is: 0 value is: [{'1': <class 'cerebros.units.units.InputUnit'>}]
0
k is: 1 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}]
1
Trying to create level 1
We think level 1's predecessors are: [0]
k is: 2 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'2': <class 'cerebros.units.units.DenseUnit'>}, {'3': <class 'cerebros.units.units.DenseUnit'>}]
2
Trying to create level 2
We think level 2's predecessors are: [0, 1]
k is: 3 value is: [{'1': <class 'cerebros.units.units.DenseUnit'>}]
3
Trying to create level 3
We think level 3's predecessors are: [0, 1, 2]
k is: 4 value is: [{'2': <class 'cerebros.units.units.DenseUnit'>}, {'4': <class 'cerebros.units.units.DenseUnit'>}]
4
Trying to create level 4
We think level 4's predecessors are: [0, 1, 2, 3]
k is:

In [27]:
print(f'Best accuracy achieved is {result}')
print(f'top-1 categorical accuracy')

Best accuracy achieved is 0.887499988079071
top-1 categorical accuracy


Evaluating the best model found

In [28]:
#best_model_found = cerebros_automl.get_best_model()
best_model_found =\
tf.keras.models.load_model(cerebros_automl.best_model_path,\
safe_mode=False)

In [29]:
#
eval_loss = tf.keras.losses.CategoricalCrossentropy()
#
eval_metrics =\
[tf.keras.metrics.TopKCategoricalAccuracy(k=1,\
            name='eval_top_1_categorical_accuracy'),
 tf.keras.metrics.TopKCategoricalAccuracy(k=5,\
            name='eval_top_5_categorical_accuracy')
]

In [30]:
best_model_found.compile(loss=eval_loss, metrics=eval_metrics)
best_model_found.summary()

Model: "NeuralNetworkFuture_0000000000000nan_tr_1_nn_materialized"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 NeuralNetworkFuture_000000  [(None, 32, 32, 3)]          0         []                            
 0000000nan_tr_1_InputLevel                                                                       
 _0000000000000000_tr_1_Inp                                                                       
 utUnit_0000000000000000_tr                                                                       
 _1_0_inp (InputLayer)                                                                            
                                                                                                  
 model_2 (Functional)        (None, 1280)                 2033136   ['NeuralNetworkFuture_00000000
                                          

In [31]:
print("Evaluating best model found ...")
print("Loss | Top-1 accuracy | Top-5 accuracy")
y_test_cat = to_categorical(y_test, 10)
best_model_found.evaluate(X_test, y_test_cat)

Evaluating best model found ...
Loss | Top-1 accuracy | Top-5 accuracy
313/313 [==============================] - 26s 74ms/step - loss: 0.4998 - eval_top_1_categorical_accuracy: 0.8766 - eval_top_5_categorical_accuracy: 0.9925


[0.4997531771659851, 0.8766000270843506, 0.9925000071525574]

Now train the best model on the entire train dataset

In [32]:
y_train_cat  = to_categorical(y_train, 10)
#
optimizer = Adam(learning_rate=0.0005)
#
loss = tf.keras.losses.CategoricalHinge()
#
metrics =\
[tf.keras.metrics.TopKCategoricalAccuracy(k=1,\
            name='eval_top_1_categorical_accuracy'),
 tf.keras.metrics.TopKCategoricalAccuracy(k=5,\
            name='eval_top_5_categorical_accuracy')
]
#
best_model_found.compile(optimizer=optimizer,
                         loss=loss,
                         metrics=metrics,
                         )

In [33]:
%%time
best_model_found.fit(X_train,
                     y_train_cat,
                     validation_split=0.35,
                     epochs=6,
                    )

Epoch 1/6
1016/1016 [==============================] - 138s 120ms/step - loss: 0.2238 - eval_top_1_categorical_accuracy: 0.8890 - eval_top_5_categorical_accuracy: 0.9911 - val_loss: 0.1865 - val_eval_top_1_categorical_accuracy: 0.9060 - val_eval_top_5_categorical_accuracy: 0.9926
Epoch 2/6
1016/1016 [==============================] - 118s 116ms/step - loss: 0.1815 - eval_top_1_categorical_accuracy: 0.9101 - eval_top_5_categorical_accuracy: 0.9944 - val_loss: 0.1666 - val_eval_top_1_categorical_accuracy: 0.9171 - val_eval_top_5_categorical_accuracy: 0.9949
Epoch 3/6
1016/1016 [==============================] - 118s 116ms/step - loss: 0.1642 - eval_top_1_categorical_accuracy: 0.9184 - eval_top_5_categorical_accuracy: 0.9948 - val_loss: 0.1642 - val_eval_top_1_categorical_accuracy: 0.9184 - val_eval_top_5_categorical_accuracy: 0.9952
Epoch 4/6
1016/1016 [==============================] - 118s 116ms/step - loss: 0.1505 - eval_top_1_categorical_accuracy: 0.9254 - eval_top_5_categorical_accu

Evaluating again

In [34]:
print("Evaluating best model found ...")
print("Loss | Top-1 accuracy | Top-5 accuracy")
best_model_found.evaluate(X_test, y_test_cat)

Evaluating best model found ...
Loss | Top-1 accuracy | Top-5 accuracy
313/313 [==============================] - 23s 74ms/step - loss: 0.1568 - eval_top_1_categorical_accuracy: 0.9217 - eval_top_5_categorical_accuracy: 0.9963


[0.15678828954696655, 0.9217000007629395, 0.9962999820709229]